In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 24.0 MB/s eta 0:00:00


In [ ]:
!pip install gdown

In [ ]:
import pandas as pd
from pulp import *
import gdown

url = "https://drive.google.com/uc?id=15BPH7-3GGWBfXPJQ3stkT6SHQECbT-pt"
output = "shifts.csv"
gdown.download(url, output, quiet=False)

df = pd.read_csv("shifts.csv", index_col=0)

Downloading...
From: https://drive.google.com/uc?id=15BPH7-3GGWBfXPJQ3stkT6SHQECbT-pt
To: /content/shifts.csv
100%|██████████| 298/298 [00:00<00:00, 973kB/s]


In [ ]:
df.head()

,Shift 1,Shift 2,Shift 3,Shift 4,Workers Required
Time Windows,,,,,
6:00 – 9:00,X,NaN,NaN,X,55.0
9:00 – 12:00,X,NaN,NaN,NaN,46.0
12:00 – 15:00,X,X,NaN,NaN,59.0
15:00 – 18:00,NaN,X,NaN,NaN,23.0
18:00 – 21:00,NaN,X,X,NaN,60.0


In [ ]:
df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

a = df.drop(index=["Wage rate per 9h shift ($)"], columns=["Workers Required"]).values
a

array([[1, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 1]], dtype=object)

In [ ]:
# number of shifts
n = a.shape[1]

# number of time windows
T = a.shape[0]

# number of workers required per time window
d = df["Workers Required"].values

# wage rate per shift
w = df.loc["Wage rate per 9h shift ($)", :].values.astype(int)

In [ ]:
# Create problem
prob = LpProblem("scheduling_workers", LpMinimize)

In [ ]:
# Decision variables
y = LpVariable.dicts("num_workers", list(range(n)), lowBound=0, cat="Integer")

In [ ]:
# Objective
prob += lpSum([w[j] * y[j] for j in range(n)])

In [ ]:
# Subject to
for t in range(T):
    prob += lpSum([a[t, j] * y[j] for j in range(n)]) >= d[t]

In [ ]:
prob.solve()

1

In [ ]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [ ]:
for shift in range(n):
    print(
        f"The number of workers needed for shift {shift} is {int(y[shift].value())} workers"
    )

The number of workers needed for shift 0 is 46 workers
The number of workers needed for shift 1 is 23 workers
The number of workers needed for shift 2 is 38 workers
The number of workers needed for shift 3 is 30 workers


In [ ]:
print(
    f"The total amount of money being paid to workers in 4 shifts are ${value(prob.objective)}",
)

The total amount of money being paid to workers in 4 shifts are $22290.0


In [ ]:
# flight schedule example

In [ ]:
import pandas as pd
import cvxpy as cp
import numpy as np
import gdown

In [ ]:
url = "https://drive.google.com/uc?id=1TPXha3ojq2e9ygGN9r2249jPieaKL1ZR"
output = "flight_schedule.csv"
gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=1TPXha3ojq2e9ygGN9r2249jPieaKL1ZR
To: /content/flight_schedule.csv
100%|██████████| 545/545 [00:00<00:00, 1.37MB/s]


'flight_schedule.csv'

In [ ]:
df = pd.read_csv("flight_schedule.csv", index_col=0, header=1)
schedule = df.iloc[:-2]
cost = df.iloc[-2]
hours = df.iloc[-1]

In [ ]:
df.head()

,1,2,3,4,5,6,7,8
Requirement (Flight),,,,,,,,
New York to Buffalo,1,0,0,1,0,0,1,0
New York to Cincinnati,0,1,0,0,1,0,0,0
New York to Chicago,0,0,1,0,0,1,0,1
Buffalo to Chicago,2,0,0,2,0,0,0,0
Chicago to Cincinnati,0,0,2,3,0,2,0,0


In [ ]:
schedule = schedule.applymap(lambda x: 1 if x >= 1 else x)

In [ ]:
schedule

,1,2,3,4,5,6,7,8
Requirement (Flight),,,,,,,,
New York to Buffalo,1,0,0,1,0,0,1,0
New York to Cincinnati,0,1,0,0,1,0,0,0
New York to Chicago,0,0,1,0,0,1,0,1
Buffalo to Chicago,1,0,0,1,0,0,0,0
Chicago to Cincinnati,0,0,1,1,0,1,0,0
Cincinnati to Pittsburgh,0,1,0,1,0,1,0,0
Cincinnati to Buffalo,0,0,1,0,1,0,0,0
Buffalo to New York,0,0,1,0,1,0,1,0
Pittsburgh to New York,0,1,0,1,0,1,0,0


In [ ]:
a = schedule.values
c = cost.values
b = np.ones(len(a))

In [ ]:
y = cp.Variable(len(c), boolean=True)

In [ ]:
# Constraint: ensure that at least one crew is assigned to each flight.

# For example, to make sure that at least one crew is assigned to the first flight, we have constraint

In [ ]:
constraints = [a @ y >= b]

In [ ]:
obj = cp.Minimize(c @ y)

In [ ]:
prob = cp.Problem(obj, constraints)

In [ ]:
prob.solve(solver=cp.GLPK_MI)

13.0

In [ ]:
print(prob.status)

optimal


In [ ]:
print(y.value)

[1. 1. 1. 0. 0. 0. 0. 0.]


In [ ]:
h = hours.values
h

array([ 396,  352, 1022,  847,  687,  531,  236,  179])

Constraints:

There must be at least one airplane per flight
The total hours of flights must be less than or equal to 1700

In [ ]:
constraints = [a @ y >= b, h @ y <= 1700]

In [ ]:
prob = cp.Problem(obj, constraints)

In [ ]:
prob.solve(solver=cp.GLPK_MI)

20.0

In [ ]:
print(y.value)

[1. 0. 0. 0. 1. 1. 0. 0.]


In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

In [ ]:
# Constants
virtual_class_duration = 0.5
in_person_class_duration = 1.0
days_per_week = 5
days_per_month = 20
in_person_classes = 50
virtual_classes = 1000
instructor_vacation = 20
instructor_travel = 60
in_person_classes_with_travel = 10
max_in_person_classes_per_week = 4
max_travel_per_month = 2

In [ ]:
# Create the LP problem
problem = LpProblem("Instructor_Scheduling", LpMinimize)

In [ ]:
# Variables
instructors_needed = LpVariable("Instructors_Needed", lowBound=1, cat='Integer')

In [ ]:
# Objective function: Minimize the number of instructors needed
problem += instructors_needed


In [ ]:

# Constraints
total_in_person_duration = in_person_classes * in_person_class_duration
total_virtual_duration = virtual_classes * virtual_class_duration

total_work_days_per_year = days_per_week * (365 - instructor_vacation - instructor_travel)
total_in_person_days = total_in_person_duration + in_person_classes_with_travel
total_virtual_days = total_virtual_duration

# Each instructor works 5 days a week
problem += (total_in_person_days + total_virtual_days) <= (instructors_needed * total_work_days_per_year)

# Each instructor can teach a maximum of 3 in-person classes per week
problem += (in_person_class_duration * max_in_person_classes_per_week * instructors_needed) >= total_in_person_days

# Each instructor can travel a maximum of 2 times per month
problem += (instructor_travel / days_per_month * instructors_needed) >= (in_person_classes_with_travel / 3)




In [ ]:
# Solve the problem
problem.solve()

# Print the results
optimal_instructors = int(instructors_needed.value())
print(f"Optimal number of instructors needed: {optimal_instructors}")

Optimal number of instructors needed: 15


In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMaximize

# Constants
virtual_class_duration = 0.5
in_person_class_duration = 1.0
days_per_week = 5
days_per_month = 20
instructor_vacation = 20
instructor_travel = 60
in_person_classes_with_travel = 40
max_in_person_classes_per_week = 3
max_travel_per_month = 2
number_of_instructors = 12

# Create the LP problem
problem = LpProblem("Class_Scheduling", LpMaximize)

# Variables
in_person_fraction = 0.25  # 25% in-person classes
virtual_fraction = 0.75     # 75% virtual classes

in_person_classes_var = LpVariable("In_Person_Classes", lowBound=0, cat='Integer')
virtual_classes_var = LpVariable("Virtual_Classes", lowBound=0, cat='Integer')

# Objective function: Maximize the total number of classes
problem += in_person_classes_var + virtual_classes_var

# Constraints
total_in_person_duration = in_person_classes_var * in_person_class_duration
total_virtual_duration = virtual_classes_var * virtual_class_duration

total_work_days_per_year = days_per_week * (365 - instructor_vacation - instructor_travel)
total_in_person_days = total_in_person_duration + in_person_classes_with_travel
total_virtual_days = total_virtual_duration

# Each instructor works 5 days a week
problem += (total_in_person_days + total_virtual_days) <= (number_of_instructors * total_work_days_per_year)

# Each instructor can teach a maximum of 3 in-person classes per week
problem += (in_person_class_duration * max_in_person_classes_per_week * number_of_instructors) >= total_in_person_days

# Each instructor can travel a maximum of 2 times per month
problem += (instructor_travel / days_per_month * number_of_instructors) >= (in_person_classes_with_travel / 3)

# Define the fractions of classes
problem += in_person_classes_var >= in_person_fraction * (in_person_classes_var + virtual_classes_var)
problem += virtual_classes_var >= virtual_fraction * (in_person_classes_var + virtual_classes_var)

# Solve the problem
problem.solve()

# Get the optimal number of classes
optimal_in_person_classes = int(in_person_classes_var.value())
optimal_virtual_classes = int(virtual_classes_var.value())

print(f"Optimal number of in-person classes: {optimal_in_person_classes}")
print(f"Optimal number of virtual classes: {optimal_virtual_classes}")


Optimal number of in-person classes: -4
Optimal number of virtual classes: -12
